<a href="https://colab.research.google.com/github/wyntermindnich/proj2AI/blob/main/P2_Refactored.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!pip install gym

!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install -U colabgymrender

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,116 kB]
Hit:12 http://ppa.launchpad.net/cra

In [2]:
import gym
import torch
import numpy as np
import torch
from random import choice
import random
from PIL import Image
import torchvision
import torchvision.transforms as transforms
#from colabgymrender.recorder import Recorder
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 1

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        #input is a image - 2d or 3d 
        #conv layer can take care of that
        #conv can be applied to any size
        #all that needs to specified is the channels
        #our input is grayscale
        #first param - input
          #num of framess passed in as 
          #input is four frames, 4 channels
          #output channels is the number of convolutional neurons
          #each neuron produces a single channel by adding them all together
          #approximate luminance by turning into grayscale!
          #the job of a convolutional layer is to learn a filter
          #if 32 filters we have 32 neurons = 32 output channels
            #ea neuron learns to look for something in the input image  
            #8x8 will produce 10x10        
        self.conv1 = nn.Conv2d(3, 32, 8, stride = 4)
        #self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64,4, stride = 2)
        self.conv3 = nn.Conv2d(64, 64,3, stride = 1)

        #we need to specify to our fully connected layer
        #the shape of output coming conv3 and how many are coming from conv3 +
        #call shape on size of x/?

        #fully connected layer
        self.fc3 = nn.Linear(64*49, 512)
        self.fc4 = nn.Linear(1*512, 10)

        
#takes image passes through image
#takes 84x84x4 image and outputs 1 x 10
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        convolved = self.conv3(x)
        
        convolved = F.relu(convolved)
        #here it should be followed by a rectifier
        #is this the same as rectified non-linearity///?
        #print(convolved.shape)
        #determine number of pixels from conv3 to call on linear
        x = x.size()

        convolved= torch.flatten(convolved)

        fullyConn1 = self.fc3(convolved)

        rectified = F.relu(fullyConn1)

        fullyConn2 = self.fc4(rectified)
        rectified = F.relu(fullyConn2)


        return fullyConn2

In [5]:
net = Net()
net = net.to("cuda")
#print(net(torch.rand(1,4,84,84) ))

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

ratings = []

# for i in net(torch.rand(1,4,84,84) ):
#   ratings.append(i)

# print(classes[ratings.index(max(ratings))])

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
      # get the inputs; data is a list of [inputs, labels]
      inputs, labels = data

      inputs = transforms.ToPILImage()(inputs[0,:,:,:])
      inputs = torchvision.transforms.Resize([84,84])(inputs)
      inputs = torchvision.transforms.ToTensor()(inputs)
      inputs = inputs.unsqueeze(0)
      # get the inputs; data is a list of [inputs, labels]
      inputs = inputs.to("cuda")
      labels = labels.to("cuda")

      # zero the parameter gradients
      optimizer.zero_grad()

      # forward + backward + optimize
      outputs = net(inputs)
      outputs = outputs.unsqueeze(0)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

        # print statistics
      running_loss += loss.item()
      if i % 2000 == 1999:    # print every 2000 mini-batches
          print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 2000))
          running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 2.306
[1,  4000] loss: 2.303
[1,  6000] loss: 2.303
[1,  8000] loss: 2.296
[1, 10000] loss: 2.267
[1, 12000] loss: 2.228
[1, 14000] loss: 2.174
[1, 16000] loss: 2.106
[1, 18000] loss: 2.085
[1, 20000] loss: 2.046
[1, 22000] loss: 1.988
[1, 24000] loss: 2.014
[1, 26000] loss: 1.949
[1, 28000] loss: 1.932
[1, 30000] loss: 1.895
[1, 32000] loss: 1.917
[1, 34000] loss: 1.889
[1, 36000] loss: 1.898
[1, 38000] loss: 1.889
[1, 40000] loss: 1.850
[1, 42000] loss: 1.830
[1, 44000] loss: 1.827
[1, 46000] loss: 1.793
[1, 48000] loss: 1.801
[1, 50000] loss: 1.752
[2,  2000] loss: 1.780
[2,  4000] loss: 1.801
[2,  6000] loss: 1.734
[2,  8000] loss: 1.723
[2, 10000] loss: 1.715
[2, 12000] loss: 1.725
[2, 14000] loss: 1.744
[2, 16000] loss: 1.736
[2, 18000] loss: 1.764
[2, 20000] loss: 1.714
[2, 22000] loss: 1.676
[2, 24000] loss: 1.664
[2, 26000] loss: 1.724
[2, 28000] loss: 1.720
[2, 30000] loss: 1.716
[2, 32000] loss: 1.681
[2, 34000] loss: 1.695
[2, 36000] loss: 1.699
[2, 38000] 

In [7]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

NameError: ignored